In [1]:

import pandas as pd
pd.set_option('display.max_columns', None)
from pathlib import Path
import json
dataset_path = Path("/mnt/i/bio/mave_curation/Findlay_VHL_SGE/")
assert dataset_path.exists()
data = pd.read_excel(dataset_path/ "raw" / "41588_2024_1800_MOESM4_ESM.xlsx",header=2)

In [2]:
data

,hg38_pos,ref,alt,cHGVS,pHGVS,function_score_final,q_value,rna_score,rna_score_day_20,delta_rna,function_class,sge_region,Intron,Exon,cDNApos,CDSpos,Chrom,conseq,oAA,nAA,CCDS,protPos,variant,cigar,edit_string,tHDR_pre,tHDR_post,tHDR_lib,tHDR_neg,tHDR_rna,tHDR_post2,tHDR_rna2,rLD2_tHDR_pre,rLD2_tHDR_lib,rLD2_tHDR_post,rLD2_tHDR_rna,rLD2_tHDR_post2,rLD2_tHDR_rna2,clinvar_simple,clinvar_group,vhldb_rcc_count,vhldb_pheo_count,vhldb_hb_count,vhldb_t1_count,vhldb_t2_count,vhldb_t2a_count,vhldb_t2b_count,vhldb_t2c_count,vhldb_entries,vhldb_homozygous_count,vhldb_t1_only,vhldb_t1_or_2b_only,vhldb_t2_only,vhldb_t2a_only,vhldb_t2b_only,vhldb_t2c_only,vhldb_rcc_only,vhldb_rcc_no_pheo,vhldb_pheo_only,vhldb_rcc_any,vhl_db_annotation,Chromophobe.Renal.Cell.Carcinoma,Renal.Clear.Cell.Carcinoma,Renal.Clear.Cell.Carcinoma.with.Sarcomatoid.Features,Papillary.Renal.Cell.Carcinoma,Unclassified.Renal.Cell.Carcinoma,Unclassified.Kidney.Renal.Cell.Carcinoma,Pheochromocytoma,Renal.Cell.Carcinoma,Pancreatic.Neuroendocrine.Tumor,Kidney.Renal.Cell.Carcinoma.Other,total_cBP_entries,total_cBP_rcc,total_cBP_other,total_cBP_ccRCC,OncoKB,CancerHotspot,Cancer_type_single,OncoKB_simple,ukb_allele_count,ukb_allele_number,ukb_allele_frequency,bravo_het_count,bravo_freq,gv2_allele_count,gv2_allele_number,gv2_allele_freq,gv3_allele_count,gv3_allele_number,gv3_allele_freq,combined_pop_count,CADD.phred,boostDM_score,boostDM_class,EVE_scores_ASM,VARITY_R,VARITY_R_LOO,VARITY_ER,VARITY_ER_LOO,REVEL,function_score_HIF1AKO,q_value_HIF1AKO,function_score_no_dab,q_value_no_dab,SpliceAI.acc.gain,SpliceAI.acc.loss,SpliceAI.don.gain,SpliceAI.don.loss,max_spliceAI,ddG.VHL.complex,ddG.VHL.only,exposure,low_rna,mis_vep_category,novel.clinvar
0,10141958,G,C,c.111G>C,p.E37D,0.227104,0.999801,0.146014,0.049020,-0.096993,Neutral,exon 1b,NaN,1/3,951.0,111.0,3,NON_SYNONYMOUS,E,D,CCDS2597.1,37.0,GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...,214M,"48-X-C,72-X-C,87-X-A",503,735,1355,1,208.0,655,246.0,426,1355,626,282.0,569,316.0,Uncertain significance,Uncertain / Conflicting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not pathogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,Absent,NaN,0.0,NaN,0.0,0.0,0.0,1.0,149368.0,0.000007,0.0,0.0,0.000000,1.0,8.052,0.005146,0.0,NaN,0.145,0.145,0.053,0.053,0.171,NaN,NaN,-0.078495,0.753713,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
1,10141958,G,T,c.111G>T,p.E37D,0.480106,0.999801,0.496812,0.314450,-0.182362,Neutral,exon 1b,NaN,1/3,951.0,111.0,3,NON_SYNONYMOUS,E,D,CCDS2597.1,37.0,GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...,214M,"48-X-T,72-X-C,87-X-A",581,923,1925,1,244.0,549,353.0,483,1925,915,512.0,638,584.0,absent,Absent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not pathogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,Absent,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,8.166,0.005146,0.0,NaN,0.145,0.145,0.053,0.053,0.171,NaN,NaN,0.289485,0.994830,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
2,10141959,T,A,c.112T>A,p.S38T,-0.065021,0.736202,-0.072035,-0.281796,-0.209761,Neutral,exon 1b,NaN,1/3,952.0,112.0,3,NON_SYNONYMOUS,S,T,CCDS2597.1,38.0,GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...,214M,"49-X-A,72-X-C,87-X-A",394,442,891,2,193.0,369,162.0,368,891,477,152.0,405,139.0,absent,Absent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not pathogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,Absent,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,6.819,0.012990,0.0,NaN,0.134,0.134,0.067,0.067,0.159,NaN,NaN,-0.231535,0.352427,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0
3,10141959,T,G,c.112T>G,p.S38A,0.235124,0.999801,-2.064347,-1.778039,0.286308,Neutral,exon 1b,NaN,1/3,952.0,112.0,3,NON_SYNONYMOUS,S,A,CCDS2597.1,38.0,GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...,214M,"49-X-G,72-X-C,87-X-A",483,498,1301,1,84.0,415,67.0,381,1301,802,28.0,506,8

In [15]:
from Bio.PDB.Polypeptide import protein_letters_3to1
protein_letters_1to3 = {v: k.title() for k, v in protein_letters_3to1.items()}
protein_letters_1to3['*'] = "Ter"
def make_hgvs_pro(s):
    return f"p.{protein_letters_1to3[s[2]]}{s[3:-1]}{protein_letters_1to3[s[-1]]}"

In [22]:
data = data.dropna(subset=['pHGVS',])
data = data.assign(hgvs_pro=data.pHGVS.apply(make_hgvs_pro)).rename(columns=dict(function_score_final='score'))

In [23]:
data

,hg38_pos,ref,alt,cHGVS,pHGVS,score,q_value,rna_score,rna_score_day_20,delta_rna,...,SpliceAI.don.gain,SpliceAI.don.loss,max_spliceAI,ddG.VHL.complex,ddG.VHL.only,exposure,low_rna,mis_vep_category,novel.clinvar,hgvs_pro
0,10141958,G,C,c.111G>C,p.E37D,0.227104,0.999801,0.146014,0.049020,-0.096993,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Glu37Asp
1,10141958,G,T,c.111G>T,p.E37D,0.480106,0.999801,0.496812,0.314450,-0.182362,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Glu37Asp
2,10141959,T,A,c.112T>A,p.S38T,-0.065021,0.736202,-0.072035,-0.281796,-0.209761,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Ser38Thr
3,10141959,T,G,c.112T>G,p.S38A,0.235124,0.999801,-2.064347,-1.778039,0.286308,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Ser38Ala
4,10141960,C,A,c.113C>A,p.S38Y,-0.324690,0.124480,0.548857,0.549808,0.000951,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,p.Ser38Tyr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,10141956,G,T,c.109G>T,p.E37*,1.197395,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Glu37Ter
2264,10141947,G,C,c.100G>C,p.G34R,1.199658,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Gly34Arg
2265,10141951,C,A,c.104C>A,p.A35D,1.452510,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Ala35Asp
2266,10141936,G,T,c.89G>T,p.G30V,1.506749,0.986207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p.Gly30Val


In [24]:
scoreset = pd.DataFrame(data.groupby('hgvs_pro').score.mean()).reset_index()

In [25]:
scoreset

,hgvs_pro,score
0,p.Ala11Pro,0.018568
1,p.Ala122Ala,-0.098126
2,p.Ala122Glu,-0.123239
3,p.Ala122Gly,-0.459489
4,p.Ala122Pro,-0.815256
...,...,...
1269,p.Val87Glu,-0.626087
1270,p.Val87Gly,-2.599586
1271,p.Val87Ile,-0.078465
1272,p.Val87Leu,0.014823


In [26]:
scoreset.to_csv(dataset_path / "scoreset.csv", index=False)

In [27]:
metadata = dict(uniprot_acc='P40337')
with open(dataset_path / "metadata.json", "w") as f:
    json.dump(metadata, f)